# Pushshift

In [ ]:
import pandas as pd
import requests
import json
import csv
import time
import datetime

In [ ]:
def getPushshiftData(query, after, before, sub):
    url = 'https://api.pushshift.io/reddit/search/submission/?title='+str(query)+'&size=1000&after='+str(after)+'&before='+str(before)+'&subreddit='+str(sub)
    #print(url)
    r = requests.get(url)
    data = json.loads(r.text)
    return data['data']

In [ ]:
#This function will be used to extract the key data points from each JSON result
def collectSubData(subm):
    #subData was created at the start to hold all the data which is then added to our global subStats dictionary.
    subData = list() #list to store data points
    title = subm['title']
    url = subm['url']
    #flairs are not always present so we wrap in try/except
    try:
        flair = subm['link_flair_text']
    except KeyError:
        flair = "NaN"    
    author = subm['author']
    sub_id = subm['id']
    score = subm['score']
    created = datetime.datetime.fromtimestamp(subm['created_utc']) #1520561700.0
    numComms = subm['num_comments']
    permalink = subm['permalink']

    #Put all data points into a tuple and append to subData
    subData.append((sub_id,title,url,author,score,created,numComms,permalink,flair))
    #Create a dictionary entry of current submission data and store all data related to it
    subStats[sub_id] = subData

In [ ]:
query = "" #Keyword(s) to look for in submissions
sub = "usmc" #Which Subreddit to search in

In [ ]:
dic = {'Jan': ['1514782800', '1517461200'], 'Feb': ['1517461201', '1519880400'], 
      'Mar': ['1519880401', '1522555200'], 'Apr': ['1522555201', '1525147200'],
      'May': ['1525147201', '1527825600'], 'Jun': ['1527825601', '1530417600'],
      'Jul': ['1530417601', '1533096000'], 'Aug': ['1533096001', '1535774400'],
      'Sep': ['1535774401', '1538366400'], 'Oct': ['1538366401', '1541044800'],
      'Nov': ['1541044801', '1543640400'], 'Dec': ['1543640401', '1546318800']}

In [ ]:
from tqdm import tqdm


#subCount tracks the no. of total submissions we collect
subCount = 0
#subStats is the dictionary where we will store our data.
subStats = {}
after = dic['Dec'][0]
before = dic['Dec'][1]
# We need to run this function outside the loop first to get the updated after variable
data = getPushshiftData(query, after, before, sub)
# Will run until all posts have been gathered i.e. When the length of data variable = 0
# from the 'after' date up until before date
while len(data) > 0: #The length of data is the number submissions (data[0], data[1] etc), once it hits zero (after and before vars are the same) end
    for submission in data:
        try:
            collectSubData(submission)
            subCount+=1
        except:
            continue
    # Calls getPushshiftData() with the created date of the last submission
    #update after variable to last created date of submission
    after = data[-1]['created_utc']
    #data has changed due to the new after variable provided by above code
    try:
        data = getPushshiftData(query, after, before, sub)
    except:
        break

count = 0
with open(f'Dec_2018_posts.txt', 'w', encoding="utf-8") as w:
    for i in tqdm(list(subStats.values())):
        count += 1
        w.write(f'{i[0][1]} \n')
print(f'{count} examples written')